In [117]:
%run runaway_functionsv3
cl = Cluster("Berkeley_97")

# cl.stars_in_region()
# cl.stars_in_region()

In [118]:
cl.fast_stars_in_region()

RA_ICRS_1,DE_ICRS_1,e_RA_ICRS,e_DE_ICRS,_r_1,HIP,TYC2,Source,rgeo,Plx,e_Plx,pmRA,pmDE,e_pmRA,e_pmDE,RUWE,Teff,logg,Gmag,BP-RP,BPmag,RPmag,RV,e_RV,b_rgeo,B_rgeo,FG,e_FG,FBP,e_FBP,FRP,e_FRP,RAVE5,RAVE6,e_Gmag,e_BPmag,e_RPmag,e_BP-RP,SkyCoord,rmRA,rmDE,v_pec
deg,deg,mas,mas,,,,,pc,mas,mas,mas / yr,mas / yr,mas / yr,mas / yr,,K,log(cm.s**-2),mag,mag,mag,mag,km / s,km / s,pc,pc,,,,,,,,,,,,,"deg,deg,pc",mas / yr,mas / yr,km / s
float64,float64,float64,float64,float64,int32,str11,int64,float64,float64,float32,float64,float64,float32,float32,float64,float64,float64,float64,float64,float64,float64,float64,float32,float64,float64,float64,float32,float32,float32,float32,float32,str1,str1,float64,float32,float32,float32,SkyCoord,float64,float64,float64
339.78276898862,59.07086926297,0.0265,0.0264,4.4837,--,--,2008431406040225664,2606.06836000,0.3659,0.0315,-3.951,-0.770,0.035,0.034,0.995,--,--,11.117977,3.769605,13.508001,9.738397,-69.25,0.34,2369.06812000,2884.74414000,672598.26706,1163,5.398e+04,605.7,1.009e+06,4436,--,--,0.003334,0.0125,0.006089,0.01859,"339.78276898862,59.07086926297,2606.06836",-1.2176428089141846,1.1141785192489624,20.387853576815186
340.11072199405,58.92710174793,0.0187,0.0209,9.3118,--,--,2008379900791042048,2509.77856000,0.3757,0.0235,-5.845,-4.092,0.025,0.027,1.039,3695.2,0.5505,11.359638,3.303004,13.357115,10.054111,-89.02,0.35,2353.26758000,2706.65967000,538383.27321,458,6.203e+04,346.3,7.542e+05,1542,--,--,0.002906,0.006673,0.004383,0.01106,"340.11072199405,58.92710174793,2509.77856",-3.1116428089141843,-2.2078214807510372,45.388586118778576
339.74109632951,58.83600736330,0.0092,0.0096,10.6625,--,--,2008378212857607680,2628.28149000,0.3736,0.0111,-4.228,-2.178,0.012,0.012,0.984,--,--,12.033627,2.239635,13.207543,10.967909,-43.07,0.24,2548.67554000,2691.39307000,289398.40372,52.27,7.119e+04,45.75,3.251e+05,80.88,--,--,0.002762,0.002876,0.003789,0.006665,"339.74109632951,58.8360073633,2628.28149",-1.4946428089141843,-0.29382148075103753,18.97672144868768
339.43148475774,58.95033027843,0.0375,0.0380,13.3423,--,--,2008409862483646080,2365.09619000,0.4191,0.0452,-0.075,-2.012,0.051,0.046,1.027,--,--,12.311062,4.622185,15.445667,10.823483,-33.77,0.33,2146.03052000,2611.00269000,224141.35144,321.3,9060,87.73,3.713e+05,1419,--,--,0.003164,0.01088,0.005612,0.01649,"339.43148475774,58.95033027843,2365.09619",2.6583571910858153,-0.1278214807510376,29.836092292894257
339.78212101782,58.86686000942,0.0101,0.0109,8.5270,--,--,2008381309540033024,2141.31763000,0.4553,0.0126,-5.181,-4.065,0.014,0.013,0.958,4666.5,1.7444,12.509231,2.744433,14.062089,11.317656,-68.77,0.26,2091.26025000,2198.20801000,186747.30108,44.38,3.24e+04,38.83,2.356e+05,122.1,--,--,0.002767,0.003079,0.003821,0.0069,"339.78212101782,58.86686000942,2141.31763",-2.4476428089141846,-2.180821480751038,33.273753811137496
339.41403033977,58.99750133848,0.0125,0.0125,13.4761,--,--,2008410201776259968,2186.04419000,0.4479,0.0151,-5.271,-4.514,0.017,0.016,0.996,--,--,12.627661,2.667293,14.122107,11.454813,-60.34,0.27,2125.72095000,2255.96118000,167448.92287,36.62,3.066e+04,41.42,2.076e+05,79.32,--,--,0.002766,0.003152,0.003802,0.006954,"339.41403033977,58.99750133848,2186.04419",-2.5376428089141845,-2.629821480751038,37.86769328936914
339.56494819769,58.99344914887,0.0102,0.0102,8.8317,--,--,2008407491661705216,2915.46533000,0.3094,0.0119,-4.332,-3.729,0.013,0.013,1.234,6899.3,2.8208,13.123951,1.392846,13.738716,12.345870,--,--,2842.50513000,3028.43872000,106014.76839,30.59,4.364e+04,44.13,9.137e+04,21.25,--,--,0.002773,0.002998,0.003788,0.006786,"339.56494819769,58.99344914887,2915.46533",-1.5986428089141844,-1.8448214807510377,33.734465330421564
339.91038508587,58.88525715423,0.0083,0.0093,7.4014,--,--,2008378625176070272,2198.47217000,0.4177,0.0105,-2.936,-5.070,0.011,0.011,0.970,--,--,13.215298,2.382625,14.493308,12.110683,-24.02,0.39,2155.42188000,2252.22729000,97460.24032,25.17,2.178e+04,22.51,1.135e+05,42.07,--,--,0.00277,0.003007,0.003801,0.006808,"339.910

In [119]:
g = cl.gießler_input()
g



TypeInput,RA,DE,Plx,e_Plx,RV,e_RV,RVdist,pmRA,e_pmRA,pmDE,e_pmDE,Source
,,,mas,mas,,,,mas / yr,mas / yr,mas / yr,mas / yr,
int32,str12,str12,float64,float32,int32,int32,int32,float64,float32,float64,float32,str21
1,22 39 23.976,59 00 16.560,0.36114121,0.0173,0,0,0,-2.733,0.099,-1.884,0.099,Berkeley_97
1,22 39 07.865,59 04 15.129,0.38371979,0.0315,0,0,0,-3.951,0.035,-0.770,0.034,2008431406040225664
1,22 40 26.573,58 55 37.566,0.39844153,0.0235,0,0,0,-5.845,0.025,-4.092,0.027,2008379900791042048
1,22 38 57.863,58 50 09.627,0.38047675,0.0111,0,0,0,-4.228,0.012,-2.178,0.012,2008378212857607680
1,22 37 43.556,58 57 01.189,0.42281578,0.0452,0,0,0,-0.075,0.051,-2.012,0.046,2008409862483646080
1,22 39 07.709,58 52 00.696,0.46700218,0.0126,0,0,0,-5.181,0.014,-4.065,0.013,2008381309540033024
1,22 37 39.367,58 59 51.005,0.45744729,0.0151,0,0,0,-5.271,0.017,-4.514,0.016,2008410201776259968
1,22 38 15.588,58 59 36.417,0.34299842,0.0119,0,0,0,-4.332,0.013,-3.729,0.013,2008407491661705216


#    RA                  DEC               Parallax        RV Maxw.        PMRA              PMDEC      ID           Remarks
# ---------------------------------------------------------------------------------------------------------------------------
1  00 14 17.75       +47 46 33.4        0.5631 0.0563     0 0 1        19.3    1.8       -19.7    1.5   B0011+47     (aus NS_no_GCs.txt, Zeile 3)
1  00 34 08.8703     -07 21 53.409        0.93   0.08     0 0 1       10.37   0.08      -11.13   0.16   B0031-07     (aus NS_no_GCs.txt, Zeile 4)
1  00 40 32.3899     +57 16 24.833        0.11   0.04     0 0 1       12.39   0.05        -5.4    0.1   B0037+56     (aus NS_no_GCs.txt, Zeile 5)
1  00 55 45.3981     +51 17 24.601        0.35   0.06     0 0 1       10.47   0.07       -17.4    0.1   B0052+51     (aus NS_no_GCs.txt, Zeile 6)
1  01 02 32.9914     +65 37 13.416         0.4   0.04     0 0 1        9.24   0.07        1.77   0.15   B0059+65     (aus NS_no_GCs.txt, Zeile 7)
1  01 08 22.5049     +66 08 34.499        0.47   0.03     0 0 1      -32.75   0.03       35.15   0.04   B0105+65     (aus NS_no_GCs.txt, Zeile 8)
1  01 08 08.347016   -14 31 50.1871        4.2    1.4     0 0 1          75      3      -152.5    1.7   J0108-1431   (aus NS_no_GCs.txt, Zeile 9)
1  01 25 01.074328   -23 27 08.1349        1.4    0.3     0 0 1       37.14   0.08        10.5    0.2   J0125-2327   (aus NS_no_GCs.txt, Zeile 10)
1  01 34 18.6824     -29 37 17.042        0.04  0.004     0 0 1          17      1          -9      2   J0134-2937   (aus NS_no_GCs.txt, Zeile 11)
1  01 47 44.6434     +59 22 03.284        0.47   0.07     0 0 1       -6.39   0.09         3.8   0.08   B0144+59     (aus NS_no_GCs.txt, Zeile 12)
1  01 51 22.7179     -06 35 02.987        0.23   0.09     0 0 1        10.7    0.1       -5.38   0.07   B0148-06     (aus NS_no_GCs.txt, Zeile 13)

In [131]:
myrun = Table.read('./Clusters/Berkeley_97/Berkeley_97_runaways_all.tsv', format='ascii.ecsv')
myrun

Temp. Est,RA_ICRS_1,DE_ICRS_1,e_RA_ICRS,e_DE_ICRS,_r_1,HIP,TYC2,Source,rgeo,Plx,e_Plx,v_pec,µ_pec,rmRA,e_rmRA,rmDE,e_rmDE,pmRA,pmDE,e_pmRA,e_pmDE,RUWE,Teff,logg,Gmag,BP-RP,BPmag,RPmag,RV,e_RV,b_rgeo,B_rgeo,FG,e_FG,FBP,e_FBP,FRP,e_FRP,RAVE5,RAVE6,e_Gmag,e_BPmag,e_RPmag,e_BP-RP
K,deg,deg,mas,mas,,,,,pc,mas,mas,km / s,mas / yr,mas / yr,mas / yr,mas / yr,mas / yr,mas / yr,mas / yr,mas / yr,mas / yr,,K,log(cm.s**-2),mag,mag,mag,mag,km / s,km / s,pc,pc,,,,,,,,,,,,
float64,float64,float64,float64,float64,float64,int32,str1,int64,float64,float64,float32,float64,float64,float64,float32,float64,float32,float64,float64,float32,float32,float64,float64,float64,float64,float64,float64,float64,float64,float32,float64,float64,float64,float32,float32,float32,float32,float32,str1,str1,float64,float32,float32,float32
14950.0,339.89581711423,58.99547917199,0.0202,0.0197,1.5205,--,--,2008384504995669248,2620.36230000,0.3560,0.0241,29.876,2.405,2.349,0.122,0.517,0.117,-0.379,-1.352,0.025,0.024,1.086,9168.0,3.7061,15.384143,1.182070,15.883185,14.701116,--,--,2461.53345000,2759.61401000,13221.76503,3.482,6055,14.81,1.044e+04,18.25,--,--,0.00277,0.003852,0.004229,0.008081
9894.0,339.82327510265,59.02791551662,0.0373,0.0353,1.6227,--,--,2008430306528315136,2914.14893000,0.3154,0.0444,21.453,1.553,-1.036,0.142,-1.157,0.136,-3.764,-3.026,0.045,0.043,1.051,--,--,16.566925,1.372125,17.177988,15.805863,--,--,2617.74390000,3300.76758000,4448.12149,2.398,1838,8.274,3774,7.525,--,--,0.002817,0.005629,0.004356,0.009985
8341.0,339.76625014965,58.97239278067,0.0263,0.0254,3.2286,--,--,2008383126302250624,2632.16675000,0.3567,0.0308,34.922,2.799,-2.688,0.131,-0.781,0.125,-5.416,-2.650,0.034,0.032,1.003,8724.0,4.1062,15.973080,1.580730,16.702312,15.121582,--,--,2389.59448000,3024.12280000,7686.25360,2.913,2848,7.975,7088,13.62,--,--,0.002786,0.004127,0.004317,0.008444
7654.0,339.72574294861,58.90086167091,0.0409,0.0403,7.3150,--,--,2008382374691897728,2437.11816000,0.3890,0.0486,48.860,4.230,-2.658,0.150,-3.290,0.143,-5.386,-5.159,0.053,0.050,1.058,8479.2,4.4474,16.878890,1.713390,17.696510,15.983120,--,--,2201.25464000,2686.80347000,3337.26628,1.812,1140,6.485,3206,8.331,--,--,0.002818,0.006778,0.004717,0.01149
7654.0,340.04434455421,59.06446992114,0.1232,0.1159,6.9950,--,--,2008385192190655616,2257.21533000,0.4704,0.1475,42.312,3.955,3.369,0.244,2.071,0.242,0.641,0.202,0.147,0.149,5.537,--,--,15.869649,1.711429,16.464157,14.752728,--,--,1733.34558000,3212.94531000,8454.48483,6.691,3546,18.75,9956,33.76,--,--,0.002886,0.006382,0.005277,0.01166
7413.0,339.86590073968,59.01069960562,0.0387,0.0372,0.6146,--,--,2008384779873564160,2957.05151000,0.3227,0.0474,54.937,3.919,3.322,0.143,2.080,0.137,0.594,0.211,0.046,0.044,1.074,7807.8,4.2979,16.695457,1.793933,17.571976,15.778043,--,--,2557.86914000,3448.42041000,3951.51579,2.257,1278,8.833,3872,6.869,--,--,0.002824,0.008004,0.004242,0.01225


In [135]:
gsources = [2008378693895483648,
            2008431406040225664,
            2008383817801137152,
            2008432058875274496,
            2008384573715370624,
            2008381034662369920,
            2008406185991730560,
            2008384504995669248,
            2008391033346185856,
            2008387421269978112,
            2008385192190655616,
            2008383126302250624,
            2008391411303287808,
            2008436319478858368,
            2008430169089361536,
            2008385020393773056,
            2008432814789510400,
            2008384814233513472,
            2008405979833246592,
            2008430306528315136,
            2008389933834808192,
            2008384779873564160,
            2008406289070887424,
            2008382374691897728,
            2008378693895483648
            ]

gtable = Table({'Source':gsources})
fs = cl.fast_stars_in_region()
fs = join(fs,gtable, keys='Source', join_type='inner')
fs.add_column(np.zeros_like(fs['BP-RP']), name='Temp. Est', index=0)
fs.write('test.tsv', format='ascii.ecsv', overwrite=True)

In [138]:
def make_gießler_input(table):
    g = Table()
    g['TypeInput'] = np.ones_like(table['e_Plx'].value).astype(int)
    g['RA'] = table['SkyCoord'].ra.to_string(unit='hourangle', sep=' ', precision=3, pad=True)
    g['DE'] = table['SkyCoord'].dec.to_string(unit='degree', sep=' ', precision=3, pad=True)
    g['Plx'] = table['rgeo'].to(u.mas, u.parallax())
    g['e_Plx'] = table['e_Plx']
    g['RV'] = np.zeros_like(table['e_Plx'].value).astype(int)
    g['e_RV'] = np.zeros_like(table['e_Plx'].value).astype(int)
    g['RVdist'] = np.zeros_like(table['e_Plx'].value).astype(int)
    g['pmRA'] = table['pmRA']
    g['e_pmRA'] = table['e_pmRA']
    g['pmDE'] = table['pmDE']
    g['e_pmDE'] = table['e_pmDE']
    g['Source'] = table['Source'].astype(str)
    return g
make_gießler_input(fs).write(f'temp.tsv', format='csv', delimiter='\t', overwrite=True)
